# Analysis of NYC public schools results in ELA and math grades 3-8.

<span style="color: red;">**If kernel can't connect to server again run command:**
*netsh winsock reset*<span>

### Generating maps of the NYC middle schools by assessment results 

<a id="TOC"></a> 
### Table of Contents
1. [Data sources and definitions](#data)
2. [Research questions](#questions)
1. [Imports: modules](#modules)
3. [Preparing layer with districts](#district)
4. [Prepare schools' layer](#schools)
5. [Generating NYC map](#nyc_map)
6. [Generating map zoomed on a district (District 15)](#map_dist_15)

<a id="data"></a> 

### Data sources and definitions

#### Data:
1. New York City grades 3-8 New York State English Language Arts and Math State Tests results 2013-2023:<br>https://infohub.nyced.org/reports/academics/test-results
2. New York City schools demographic data:<br>https://data.cityofnewyork.us/Education/2017-18-2021-22-Demographic-Snapshot/c7ru-d68s/about_data
2. NYS schools locations:<br>
https://data.gis.ny.gov/maps/b6c624c740e4476689aa60fdc4aacb8f/about
2. New York City school districts boundaries:<br>https://data.cityofnewyork.us/Education/School-Districts/r8nu-ymqj
3. Citywide or Boroughwide status:
<br>https://www.nycschoolhelp.com/borowide-citywide-middle-schools

#### Definitions of Performance Levels for the 2023 Grades 3-8 English Language Arts and Mathematics Tests  

**NYS Level 1**: Students performing at this level are below proficient in standards for their grade. They may demonstrate limited knowledge, skills, and practices embodied by the Learning Standards that are considered insufficient for the expectations at this grade. 

**NYS Level 2**: Students performing at this level are partially proficient in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered partial but insufficient for the expectations at this grade. Students performing at Level 2 are considered on track to meet current New York high school graduation requirements but are not yet proficient in Learning Standards at this grade. 

**NYS Level 3**: Students performing at this level are proficient in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered sufficient for the expectations at this grade.  

**NYS Level 4**: Students performing at this level excel in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered more than sufficient for the expectations at this grade.  

*Source: NYSED, 2023, https://www.p12.nysed.gov/irs/ela-math/2023/ela-math-score-ranges-performance-levels-2023.pdf*

<a id="questions"></a> 
### Questions
*1. How to compare the schools?*
<br>In this analysis, we choose the sum of shares of students with level 4 test resulsts in state math and ELA test as comparison variable. The sum can be between 0 and 2. This indicator is selected to cover both subjects.
ALternatively, the indicator can be sum of shares of students with levels 3+4 test results in math and ELA. The notebook would be needed to changed accordingly.
<br><br>
*2. How the test results changed?*
<br>Compare last year test results in a school with the school 10-year average as percentage of average:
<br> school_change = (school_current_year - school_10year_average)
<br> citywide_change = (city_current_year - city_10year_average)
<br> relative_school_change = school_change - citywide_change
<br><br>
*3. How good the school is?* 
<br>Last three testing period results (2019, 2022, 2023) are different for some schools: due to COVID disruptions, testing procedures changes, in Destrict 15 due to admission rules changed. Therefore average 10 years scores do not reflect well schools situation now. Results for these 3 last testing years are taken instead.
<br><br>
*3. Is the school citywide or borowide?*
<br><br>
*4. Diversity?*
<br><br>
*5. School size?*

#### About this notebook

- The notebook '*1._NYC_data_processing_by_schools.ipynb*' contains the steps for the processing data on state testing of NYC public middle schools. 
- The notebook '*2._NYC_ELA_math_data processing_by_districts.ipynb*' contains steps to process district-wide data for NYC public middle schools.
- This notebook '*3._Generating_NYC_map_by_public_schools.ipynb*' contains code to generate the maps from the processed data.
- The map is available at: https://nycmsmap.netlify.app.

<a id="modules"></a> 
#### Imports: modules

In [1]:
import os
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Search
# from shapely.geometry import Point
from tqdm import tqdm

pd.set_option('display.float_format', '{:.3f}'.format)

In [3]:
basePath = r"G:\My Drive\Kids\NYC_schools_mapped"
dataFolder = r"raw_data"
outputFolder = r"processed_data"

Below mapping is done with the __[Folium library](https://python-visualization.github.io/folium/latest/)__

<a id="district"></a>
#### Preparing layer with districts

In [4]:
# # Read file with district wide math test results to add to the map
# DistrictMathFile = "DistrictsMSmathNorm2025.xlsx"
# DistrictMathPath = os.path.join(basePath, outputFolder, DistrictMathFile)
# DistrictMSMathData = pd.read_excel(DistrictMathPath)
# print(DistrictMSMathData.head(5))

# # Read file with district wide ELA test results to add to the map
# DistrictELAFile = "DistrictsMSELANorm2025.xlsx"
# DistrictELAPath = os.path.join(basePath, outputFolder, DistrictELAFile)
# DistrictMSELAData = pd.read_excel(DistrictELAPath)
# print(DistrictMSELAData.head(5))

# Read file with district wide test results to add to the map if option 2 was used in the district notebook
DistrictFile = "DistrictsMSNorm2025.xlsx"
DistrictPath = os.path.join(basePath, outputFolder, DistrictFile)
DistrictAllData = pd.read_excel(DistrictPath)
print(DistrictAllData.head(5))

## Read GeoJSON into dataframe
DistrictsFile = 'School Districts 2024.geojson'
NYCDistrictsPath = os.path.join(basePath, dataFolder, DistrictsFile)
NYCDistrictsData = gpd.read_file(NYCDistrictsPath)

   Unnamed: 0  District  Year  Level 1 Math  Level 2 Math  Level 3 Math  \
0           0         1  2025         0.202         0.180         0.253   
1           1         2  2025         0.137         0.143         0.305   
2           2         3  2025         0.186         0.186         0.269   
3           3         4  2025         0.288         0.270         0.276   
4           4         5  2025         0.349         0.234         0.272   

   Level 4 Math  Level 1 ELA  Level 2 ELA  Level 3 ELA  Level 4 ELA  \
0         0.365        0.173        0.202        0.271        0.353   
1         0.416        0.104        0.147        0.295        0.454   
2         0.359        0.148        0.192        0.280        0.380   
3         0.165        0.246        0.274        0.261        0.219   
4         0.145        0.255        0.264        0.279        0.201   

   Level 4 Math+Ela  
0             0.718  
1             0.869  
2             0.739  
3             0.385  
4           

In [ ]:
# # Renaming the colums to contains subject name

# mathColumns = {'# Level 1':'# Level 1 Math','# Level 2':'# Level 2 Math', '# Level 3':'# Level 3 Math','# Level 4':'# Level 4 Math'}
# DistrictMSMathData.rename(columns = mathColumns, inplace = True) 
# print(DistrictMSMathData.head())

# ELAColumns = {'# Level 1':'# Level 1 ELA','# Level 2':'# Level 2 ELA', '# Level 3':'# Level 3 ELA','# Level 4':'# Level 4 ELA'}
# DistrictMSELAData.rename(columns = ELAColumns, inplace = True)
# DistrictMSELAData.head()

In [ ]:
# # Merging district-wide resutls into single dataframe

# DistrictAllData = pd.merge(DistrictMSMathData, DistrictMSELAData, on = ['Year', 'District'], how = 'inner')

In [ ]:
# # Adding the column to make district chloropleth layer

# DistrictAllData['# Level 4 Math+ELA'] = DistrictAllData['# Level 4 Math'] + DistrictAllData['# Level 4 ELA']

In [ ]:
# # Checking the dataframe

# DistrictAllData.info()

<a id="schools"></a>
#### Prepare schools' layer

In [5]:
# Read saved GeoJSON with average tests results and plots
AVGTestsPlotFile = 'schoolDataPlots2013-2025_new.geojson'
AVGTestsPlotPath = os.path.join(basePath, outputFolder,AVGTestsPlotFile)
NYCSchoolsAVGData = gpd.read_file(AVGTestsPlotPath)

In [6]:
NYCSchoolsAVGData.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 487 entries, 0 to 486
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   ATS                            487 non-null    object  
 1   Building_C                     487 non-null    object  
 2   Location_C                     487 non-null    object  
 3   Name                           487 non-null    object  
 4   Geographic                     487 non-null    int32   
 5   Latitude                       487 non-null    float64 
 6   Longitude                      487 non-null    float64 
 7   DBN                            487 non-null    object  
 8   School Name                    487 non-null    object  
 9   10yrs avg Lvl 1 Math           473 non-null    float64 
 10  10yrs avg Lvl 2 Math           473 non-null    float64 
 11  10yrs avg Lvl 3 Math           473 non-null    float64 
 12  10yrs avg Lvl 4 Math        

<a id="nyc_map"></a>
#### Generating NYC map

In [7]:
# Prepare legend

legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */
                 ">
                   <div style="display:inline-block; text-align: justify;"><i class="fa fa-circle" style="border:0.5px solid #54B96D; color:green; border-radius:50%; display:inline-block;"></i><span style="margin-left: 5px;"> NYC public schools</span><br>
                   Circle size is proportional to the sum of the average shares of students scoring at the highest level (level 4) on state ELA and math tests over the years 2019 to 2025.</div>
                   <br>
                   <div><i class="fa fa-circle" style="border:2px solid yellow; color:green; border-radius:50%; display:inline-block;"></i><span style="margin-left: 5px;">Citiwide priority </span></div>
                   <div><i class="fa fa-circle" style="border:2px solid #1ff2ef; color:green; border-radius:50%; display:inline-block;"></i><span style="margin-left: 5px;">Brooklyn residents priority</span></div>
                   <br>
                   <div style="display:inline-block; text-align: justify;"><b>School Math+ELA % Level 4 2025 - 12 years average</b> - this line in the pop-up shows difference between the sum of the shares of the level 4 results in math and ELA for the school and their average sum for years 2013-2025 for this school. </div>
                   <div style="display:inline-block; text-align: justify;"><b>Citywide Math+ELA % Level 4 2025 - 12 years average</b>- this line in the pop-up shows citywide difference between the sum of the shares of the level 4 results in math and ELA for the middle schools and their citiwide average sum for years 2013-2025.</div>
                   <div style="display:inline-block; text-align: justify;">The two shows how the school performs comparing to the citywide trend.</div>
                   
      </div>
     '''

In [8]:
# Prepare map introduction

intro_html = """
     <div style="position: fixed; 
                 top: 10px; left: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="margin-bottom: 10px;">
                       <span style = "font-weight:bold; font-size: 1.5em;">New York City Public Middle Schools
                       </span>
                   </div>  
                   <div style="flex-grow: 1; text-align: justify;">The map is part of the project to map academic results of middle schools in New York and New Jersey. See more:
                   <br>
                   <ul>
                       <li><a href = "https://github.com/Solirinai/NY_schools_maps" target="_blank">Github repository with code and data sources.<a></li>
                       <li><a href = "https://nysmsmap.netlify.app" target="_blank">Map of New York State public and charter middle schools.<a></li>
                       <li><a href = "https://njmsmap.netlify.app" target="_blank">Map of New Jersey public and charter middle schools.</a></li>
                   </ul>
                   </div>                 
                 
      </div>
"""

In [9]:
# Prepare click instructions

intro_click = '''<div style="position: fixed; 
                 top: 150px; left: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="flex-grow: 1;">
                    Click on a circle to see historical test results for middle school grades (grades 6-8) for the school.
                   </div>                                     
      </div>
     '''

In [10]:
# Prepare notes and explanations

notes = '''<div style="position: fixed; 
                 bottom: 50px; right: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="margin-bottom: 10px;">
                   <span style = "font-weight:bold; font-size: 1.0em;">Limitations:</span>
                   </div> 
                   <div style="flex-grow: 1; margin-bottom: 10px;">
                       <div>
                       <ul>
                           <li style="display:inline-block; text-align: justify;"> Due to inconsistent spelling of school names in the source data, certain schools have multiple overlapping circles on the map, with pop-ups displaying data for different years.</li>
                       </ul>
                       </div> 
                       <div>
                       <span style = "font-weight:bold; font-size: 1.0em;">Data</span>
                       <ul>
                       <li><a href = 'https://infohub.nyced.org/reports/academics/test-results'>NYS Test Results, 2013-25, New York City Department of Education</a>.</li>
                       <li><a href = "https://data.cityofnewyork.us/Education/2017-18-2021-22-Demographic-Snapshot/c7ru-d68s/about_data">2017-18 - 2021-22 Demographic Snapshot, NYC Open Data</a>.</li>
                       <li><a href = "https://data.gis.ny.gov/maps/b6c624c740e4476689aa60fdc4aacb8f/about">NYS Schools locations, NYS GIS Clearinghouse</a>.</li>
                       <li><a href = "https://data.cityofnewyork.us/Education/School-Districts/r8nu-ymqj">New York City School Districts Boundaries, NYC Open Data</a>.</li>
                       <li><a href = "https://www.nycschoolhelp.com/borowide-citywide-middle-schools">Citywide or Boroughwide status, NYC School Help</a>.</li>
                       </ul> 
                       </div>
                       <div>Map developed by <a href = "https://github.com/Solirinai" target="_blank">Solirinai</a>.</div>
                   </div>
      </div>
     '''

In [14]:
lazy_script = """
<script>
(function() {
  function activateLazyImgs(container) {
    if (!container) return;
    container.querySelectorAll('img[data-src]').forEach(function(img){
      if (!img.getAttribute('src')) {
        img.setAttribute('src', img.getAttribute('data-src'));
      }
    });
  }
  // For already-rendered popups in notebooks
  document.addEventListener('DOMContentLoaded', function() {
    document.querySelectorAll('.leaflet-popup-content').forEach(activateLazyImgs);
  });
  // For Leaflet runtime popups
  var maps = document.querySelectorAll('.folium-map');
  maps.forEach(function(mapDiv){
    // Leaflet attaches map instance to mapDiv._leaflet_id, but we can listen globally:
    document.addEventListener('popupopen', function(e){
      var content = e && e.popup && e.popup.getContent && e.popup.getContent();
      if (content && content instanceof HTMLElement) {
        activateLazyImgs(content);
      }
    });
  });
})();
</script>
"""


In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.output_scroll { height: auto !important; max-height: 1500px; }</style>"))

print("Generating map ...")

# Create a map object, centered at NYC
testMS_map = folium.Map(location=[40.6839, -73.9026], zoom_start=11, tiles="cartodb positron")


# Adding the districts layer to the map
districts = folium.Choropleth(
    geo_data = NYCDistrictsData,
    data = DistrictAllData[DistrictAllData['Year'] == 2025],
    columns = ['District','# Level 4 Math+ELA'],
    key_on = "feature.properties.school_dist",
    fill_color = "BuPu",
    fill_opacity = 0.5,
    line_opacity=0.3,
    nan_fill_color="white",
    legend_name = 'Sum of percentages of middle school test takers with Level 4 result in Math and ELA, 2025',
    name = "School districts",
).add_to(testMS_map) 

#Adding popup to the layer with districts
folium.GeoJsonPopup(fields=['school_dist'], 
                      aliases=["District:"],
                     ).add_to(districts.geojson)
    
# Define function to make markers size proportional to the '4yrs avg Lvl 4 Math+Ela' column

def my_style(x):
    level4 = x['properties']['5yrs avg Lvl 4 Math+Ela']
    openTo = x['properties']['Open to']
    color = 'yellow' if openTo == 'Citywide'  else '#1ff2ef' if openTo == 'Brooklyn' else '#54B96D'
    weight = 2 if openTo == 'Citywide'  else 2 if openTo == 'Brooklyn' else 0.5
    if level4 is None:
        level4 = 0
    #print(level4)
    return {
        "radius": (level4)*400,
        "color": color,
        "weight": weight
    }  

# Function to create iframe for a given row
def create_iframe(row):    
    html =  '<strong>{0}:</strong> {1}<br><strong>{2}:</strong> {3}<br><strong>{4}:</strong> {5}<br>\
    <strong>{6}:</strong> {7}<br><strong>{8}:</strong> {9}<br><strong>{10}:</strong> {11}<br>\
    <strong>{12}:</strong>{13} <br><img src="data:image/png;base64,{14}"><br>\
    <img src="data:image/png;base64,{15}"<br> <img src="data:image/png;base64,{16}">'.format(
        'School Name', row['LEGAL_NAME'],
        'School Number', row['DBN'],
        'Total Enrollment, students', row['Total Enrollment'],
        'Level 4 share Avg 2019-2025 Math', round(row['4yrs avg Lvl 4 Math'], 2), 
        'Level 4 share Avg 2019-2025 ELA', round(row['4yrs avg Lvl 4 ELA'], 2),
        'School Math+ELA % Level 4 2025 - 11 years average', round(row['2025-12yAVG'], 3),
        'Citywide Math+ELA % Level 4 2025 - 11 years average', ' 0.136',
        row['plot Math'], row['plot ELA'], row['Dvst_chart'])
    return folium.IFrame(html, width=500, height=450)



# Create a FeatureGroup to collect all GeoJSON layers that are individual schools
feature_group = folium.FeatureGroup(name="NYC public middle schools").add_to(testMS_map)

searchSchool = Search(
    layer=feature_group,
    geom_type="Point",
    placeholder="Enter school name",
    collapsed=False,
    search_label="LEGAL_NAME",
    search_zoom = '14',
    position='topright',
).add_to(testMS_map)


# Iterate over the geodataframe and add a pop-up to each feature
for _, row in tqdm(NYCSchoolsAVGData.iterrows(), total = len(NYCSchoolsAVGData)):
    iframe = create_iframe(row)
        
    data = gpd.GeoDataFrame(row.to_frame().T, crs=NYCSchoolsAVGData.crs)
    
    folium.GeoJson(
    data,
    marker = folium.Circle(radius=10, fill_color='green', fill_opacity=1.0, color="green", weight=0.5),
    popup = folium.Popup(iframe),
    style_function = my_style, 
    control = False    
    ).add_to(feature_group)      
    
# Adding layer control
folium.LayerControl().add_to(testMS_map)    

# Add the HTML to the map using a feature group
testMS_map.get_root().html.add_child(folium.Element(legend_html))
testMS_map.get_root().html.add_child(folium.Element(intro_html))
testMS_map.get_root().html.add_child(folium.Element(intro_click))
testMS_map.get_root().html.add_child(folium.Element(notes))


map_html = testMS_map._repr_html_() 


Generating map ...


100%|████████████████████████████████████████████████████████████████████████████████| 916/916 [00:43<00:00, 20.85it/s]


<a id="map_dist_15"></a>
#### Generating map zoomed on a district (District 15)

The code is zoomed on a district. Since folium library does not allow dynemic markers sizing, the markers are smaller in this version to avoid overlapping of the markers and allowing for more detailed schools view.

In [15]:
from IPython.display import display, HTML

display(HTML("<style>.output_scroll { height: auto !important; max-height: 1500px; }</style>"))

print("Generating map ...")

# Create a map object, centered at NYC
# testMS_map_d = folium.Map(location=[40.666591, -73.995518], zoom_start=13, tiles="cartodb positron")
testMS_map_d = folium.Map(location=[40.6839, -73.9026], zoom_start=12, tiles="cartodb positron")
   
# Define function to make markers size proportional to the '3yrs avg Lvl 4 Math+Ela' column

def my_style(x):
    level4 = x['properties']['3yrs avg Lvl 4 Math+Ela']
    openTo = x['properties']['Open to']
    color = 'yellow' if openTo == 'Citywide'  else '#1ff2ef' if openTo == 'Brooklyn' else '#54B96D'
    weight = 2 if openTo == 'Citywide'  else 2 if openTo == 'Brooklyn' else 0.5
    if level4 is None:
        level4 = 0
    #print(level4)
    return {
        "radius": (level4)*150,
        # "radius": (level4)*200,
        "color": color,
        "weight": weight
    }  

# Adding the district layer to the map
districts = folium.Choropleth(
    geo_data = NYCDistrictsData,
    data = DistrictAllData[DistrictAllData['Year'] == 2025],
    columns = ['District','Level 4 Math+Ela'],
    key_on = "feature.properties.SchoolDist",
    fill_color = "BuPu",
    fill_opacity = 0.5,
    line_opacity=0.3,
    nan_fill_color="white",
    legend_name = 'Sum of percentages of middle school test takers with Level 4 result in Math and ELA, 2024',
    name = "School districts"
).add_to(testMS_map_d)

#Adding popup to the layer with districts
folium.GeoJsonPopup(fields=['SchoolDist'], 
                      aliases=["District:"],
                     ).add_to(districts.geojson)

# Function to create iframe for a given row in a geodataframe
def create_iframe(row):    
    html =  '<strong>{0}:</strong> {1}<br><strong>{2}:</strong> {3}<br><strong>{4}:</strong> {5}<br>\
    <strong>{6}:</strong> {7}<br><strong>{8}:</strong> {9}<br><strong>{10}:</strong> {11}<br>\
    <strong>{12}:</strong>{13} <br><img data-src="data:image/png;base64,{14}"><br>\
    <img data-src="data:image/png;base64,{15}"><br> <img data-src="data:image/png;base64,{16}">'.format(
        'School Name', row['Name'],
        'School Number', row['DBN'],
        'Total Enrollment, students', row['Total Enrollment'],
        'Level 4 share Avg 2023-2025 Math', round(row['3yrs avg Lvl 4 Math'], 2), 
        'Level 4 share Avg 2023-2025 ELA', round(row['3yrs avg Lvl 4 ELA'], 2),
        'School Math+ELA % Level 4 2025 - 10 years average', round(row['2025-10yAVG'], 3),
        'Citywide Math+ELA % Level 4 2025 - 10 years average', ' 0.136',
        row['plot Math'], row['plot ELA'], row['Dvst_chart'])
    return folium.IFrame(html, width=500, height=450)

# Create a FeatureGroup to collect all GeoJSON layers that are individual schools
feature_group = folium.FeatureGroup(name="NYC public middle schools").add_to(testMS_map_d)

searchSchool = Search(
    layer=feature_group,
    geom_type="Point",
    placeholder="Enter school name",
    collapsed=False,
    search_label="Name",
    search_zoom = '14',
    position='topright',
).add_to(testMS_map_d)

# Creating the schools "layer"
# Iterate over the GeoDataFrame and add a popup to each feature
for _, row in tqdm(NYCSchoolsAVGData.iterrows(), total = len(NYCSchoolsAVGData)):
    iframe = create_iframe(row)
        
    data = gpd.GeoDataFrame(row.to_frame().T, crs=NYCSchoolsAVGData.crs)
    
    folium.GeoJson(
    data,
    marker = folium.Circle(radius=10, fill_color='green', fill_opacity=1.0, color="green", weight=0.5),
    popup = folium.Popup(iframe),
    style_function = my_style, 
    control = False    
    #zoom_on_click = True,    
).add_to(feature_group)    
        
# Adding layer control      
folium.LayerControl().add_to(testMS_map_d)    

#Adding legend to the map

# Add the HTML to the map using a feature group
testMS_map_d.get_root().html.add_child(folium.Element(lazy_script))
testMS_map_d.get_root().html.add_child(folium.Element(legend_html))
testMS_map_d.get_root().html.add_child(folium.Element(intro_html))
testMS_map_d.get_root().html.add_child(folium.Element(intro_click))
testMS_map_d.get_root().html.add_child(folium.Element(notes))
# testMS_map_d.get_root().html.add_child(folium.Element(notes))
  
# # Display the map in the notebook in desired
# testMS_map_d

# Save map to html
mfile = 'NYCMSmap_small_circles_2025.html'
mpath = os.path.join(basePath, outputFolder, mfile)
print(f'Saving map to file {mpath} ...')
map_html = testMS_map_d.save(mpath)
# map_html = testMS_map_d._repr_html_()
print("Saved.")   

Generating map ...


100%|██████████| 487/487 [00:09<00:00, 50.70it/s]


Saving map to file G:\My Drive\Kids\NYC_schools_mapped\processed_data\NYCMSmap_small_circles_2025.html ...
Saved.


In [11]:
# Check IPython version
import IPython
print(f"IPython version: {IPython.__version__}")



IPython version: 9.6.0
